In [21]:
# imports
from scipy.io import readsav
from scipy import interpolate
sys.path.append(os.path.abspath('/Users/xavier/local/Python/PYPIT/src/'))

from linetools import utils as ltu
from linetools.spectra import xspectrum1d

import armsgs

import ardebug
debug = ardebug.init()
last_updated = "2 May 2016"
version = '0.1'
msgs = armsgs.get_logger((None, debug, last_updated, version, 1))
import arwave as arwv
import arutils

try:
    from xastropy.xutils import xdebug as debugger
except:
    import pdb as debugger

# Load 1D spectrum

In [2]:
spec1d_fil = '/Users/xavier/PYPIT/Tiffany/lris_red/Science/spec1d_OFF_J1044p6306_LRISr_2016Feb16T112439.fits'
hdu = fits.open(spec1d_fil)
hdu.info()

Filename: /Users/xavier/PYPIT/Tiffany/lris_red/Science/spec1d_OFF_J1044p6306_LRISr_2016Feb16T112439.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      11   ()              
1    O983-S8787-D01-I0022  BinTableHDU     21   2048R x 6C   [D, D, K, D, D, D]   
2    O900-S8787-D01-I0022  BinTableHDU     27   2048R x 9C   [D, D, K, D, D, D, D, D, D]   
3    O401-S8787-D01-I0022  BinTableHDU     27   2048R x 9C   [D, D, K, D, D, D, D, D, D]   
4    O023-S8787-D01-I0022  BinTableHDU     27   2048R x 9C   [D, D, K, D, D, D, D, D, D]   
5    O239-S1470-D02-I0022  BinTableHDU     27   2048R x 9C   [D, D, K, D, D, D, D, D, D]   
6    O652-S1470-D02-I0022  BinTableHDU     27   2048R x 9C   [D, D, K, D, D, D, D, D, D]   
7    O809-S1470-D02-I0022  BinTableHDU     27   2048R x 9C   [D, D, K, D, D, D, D, D, D]   


In [3]:
h2_obj = Table(hdu[5].data)

In [4]:
#debugger.xplot(h2_obj['box_sky'])

## Run Algorithm

In [16]:
slf = arutils.dummy_self(pypitdir=os.getenv('PYPIT'))
slf._pixcen = np.zeros(10)
slf._lordloc = 0
slf._rordloc = 0
slf._argflag['arc'] = {}
slf._argflag['arc']['calibrate'] = {}
slf._argflag['arc']['calibrate']['detection'] = 5.
slf._argflag['arc']['calibrate']['nfitpix'] = 7.
#
slf._argflag['reduce'] = {}
slf._argflag['reduce']['flexure'] ={}
slf._argflag['reduce']['flexure']['spectrum'] = None
slf._argflag['reduce']['flexure']['maxshift'] = 20.
#
slf._argflag['run']['spectrograph'] = 'LRISr'
#
msgs._debug['flexure'] = True

In [11]:
# Load archive
reload(arwv)
arx_file, arx_spec = arwv.flexure_archive(slf, 1)

[INFO]    :: Using paranal_sky.fits file for Sky spectrum
/Users/xavier/local/Python/linetools/linetools/spectra/xspectrum1d.py:92: UserWarning: Assuming wavelength unit is Angstroms
  warnings.warn("Assuming wavelength unit is Angstroms")


In [19]:
obj_sky = xspectrum1d.XSpectrum1D.from_tuple((h2_obj['box_wave'], h2_obj['box_sky']))

In [18]:
reload(arwv)
flex_dict = arwv.flex_shift(slf, 1, obj_sky, arx_spec)

[WARNING] :: If we use Paranal, cut down on wavelength early on
[INFO]    :: Detecting lines
[INFO]    :: Extracting an approximate arc spectrum at the centre of the chip
[INFO]    :: Detecting the strongest, nonsaturated arc lines
[INFO]    :: Detecting lines
[INFO]    :: Extracting an approximate arc spectrum at the centre of the chip
[INFO]    :: Detecting the strongest, nonsaturated arc lines
[INFO]    :: Resolution of Archive=39623.9 and Observation=2094.66
[INFO]    :: Flexure correction of 0.0350787 pixels


> /Users/xavier/local/Python/PYPIT/src/arwave.py(137)flex_shift()
-> flex_dict = dict(polyfit=fit, shift=shift, subpix=subpix_grid,
(Pdb) c


## Shift wavelengths

In [22]:
x = np.linspace(0., 1., obj_sky.npix)
f = interpolate.interp1d(x, obj_sky.wavelength.value, bounds_error=False, fill_value="extrapolate")
twave = f(x+flex_dict['shift']/(obj_sky.npix-1))
new_sky = xspectrum1d.XSpectrum1D.from_tuple((twave, obj_sky.flux))

## Compare to Paranal

In [23]:
pfile = '/Users/xavier/local/Python/PYPIT/data/sky_spec/paranal_sky.fits'
phdu = fits.open(pfile)

In [24]:
pwave = phdu[0].data
pflux = phdu[1].data

In [25]:
debugger.xplot(new_sky.wavelength, h2_obj['box_sky'], xtwo=pwave, ytwo=pflux*8)